In [1]:
import os, json
from datetime import datetime
from pyspark.sql.functions import col, current_timestamp, input_file_name, to_date, unix_timestamp

INBOX = "/home/jovyan/work/data/inbox"
OUTBOX = "/home/jovyan/work/data/outbox/trips_enriched.parquet"
LOOKUP = "/home/jovyan/work/data/taxi_zone_lookup.parquet"
STATE = "/home/jovyan/work/state/manifest.json"

os.makedirs("/home/jovyan/work/state", exist_ok=True)
os.makedirs("/home/jovyan/work/data/outbox", exist_ok=True)

def load_manifest(path):
    if not os.path.exists(path):
        return {"processed_files": []}
    with open(path, "r") as f:
        return json.load(f)

def save_manifest(path, manifest):
    with open(path, "w") as f:
        json.dump(manifest, f, indent=2)

manifest = load_manifest(STATE)
processed = set(x["filename"] for x in manifest["processed_files"])

all_files = sorted([f for f in os.listdir(INBOX) if f.endswith(".parquet")])
new_files = [f for f in all_files if f not in processed]

all_files, new_files


(['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet'], [])

In [2]:
import os

INBOX = "/home/jovyan/work/data/inbox"

os.listdir(INBOX)


['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet']

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Project1-ETL").getOrCreate()

df = spark.read.parquet(
    "/home/jovyan/work/data/inbox/yellow_tripdata_2025-01.parquet"
)

df.show(5)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [4]:
df.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [5]:
df.count()

3475226

In [6]:
df_all = spark.read.parquet("/home/jovyan/work/data/inbox/*.parquet")

df_all.count()

7052769

In [7]:
df_all.select(
    "passenger_count",
    "trip_distance",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).show(10)

+---------------+-------------+--------------------+---------------------+
|passenger_count|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------------+-------------+--------------------+---------------------+
|              1|          1.6| 2025-01-01 00:18:38|  2025-01-01 00:26:59|
|              1|          0.5| 2025-01-01 00:32:40|  2025-01-01 00:35:13|
|              1|          0.6| 2025-01-01 00:44:04|  2025-01-01 00:46:01|
|              3|         0.52| 2025-01-01 00:14:27|  2025-01-01 00:20:01|
|              3|         0.66| 2025-01-01 00:21:34|  2025-01-01 00:25:06|
|              2|         2.63| 2025-01-01 00:48:24|  2025-01-01 01:08:26|
|              0|          0.4| 2025-01-01 00:14:47|  2025-01-01 00:16:15|
|              0|          1.6| 2025-01-01 00:39:27|  2025-01-01 00:51:51|
|              0|          2.8| 2025-01-01 00:53:43|  2025-01-01 01:13:23|
|              1|         1.71| 2025-01-01 00:00:02|  2025-01-01 00:09:36|
+---------------+--------

In [8]:
from pyspark.sql.functions import col

df_clean = df_all.filter(
    (col("passenger_count") > 0) &
    (col("trip_distance") > 0) &
    (col("tpep_dropoff_datetime") > col("tpep_pickup_datetime"))
)

In [9]:
total_before = df_all.count()
total_after = df_clean.count()

total_before, total_after


(7052769, 5579927)

In [10]:
bad_rows = df_all.subtract(df_clean)

bad_rows.select(
    "passenger_count",
    "trip_distance",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).show(5)


+---------------+-------------+--------------------+---------------------+
|passenger_count|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------------+-------------+--------------------+---------------------+
|              1|          0.0| 2025-01-01 11:50:50|  2025-01-01 11:51:06|
|              1|          0.0| 2025-01-01 11:10:14|  2025-01-01 11:10:26|
|              0|          2.8| 2025-01-01 12:07:26|  2025-01-01 12:17:45|
|              1|          0.0| 2025-01-01 15:49:46|  2025-01-01 15:49:52|
|              0|          1.4| 2025-01-02 10:54:05|  2025-01-02 11:05:11|
+---------------+-------------+--------------------+---------------------+
only showing top 5 rows



In [11]:
df_dedup = df_clean.dropDuplicates([
    "VendorID",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID"
])

In [12]:
count_clean = df_clean.count()
count_dedup = df_dedup.count()

count_clean, count_dedup

(5579927, 5486133)

In [13]:
LOOKUP = "/home/jovyan/work/data/taxi_zone_lookup.parquet"

zones = spark.read.parquet(LOOKUP)

zones.show(5)
zones.printSchema()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [14]:
from pyspark.sql.functions import broadcast

zones_pickup = (
    zones
    .select(col("LocationID").alias("PULocationID"),
            col("Zone").alias("pickup_zone"))
)

df_enriched = df_dedup.join(
    broadcast(zones_pickup),   # lookup é pequeno → melhora performance
    on="PULocationID",
    how="left"
)

In [15]:
zones_dropoff = (
    zones
    .select(col("LocationID").alias("DOLocationID"),
            col("Zone").alias("dropoff_zone"))
)

df_enriched = df_enriched.join(
    broadcast(zones_dropoff),
    on="DOLocationID",
    how="left"
)

In [16]:
df_enriched.select(
    "PULocationID", "pickup_zone",
    "DOLocationID", "dropoff_zone"
).show(10, truncate=False)


+------------+---------------------+------------+-----------------------------+
|PULocationID|pickup_zone          |DOLocationID|dropoff_zone                 |
+------------+---------------------+------------+-----------------------------+
|142         |Lincoln Square East  |263         |Yorkville West               |
|140         |Lenox Hill East      |50          |Clinton West                 |
|239         |Upper West Side South|143         |Lincoln Square West          |
|142         |Lincoln Square East  |239         |Upper West Side South        |
|263         |Yorkville West       |249         |West Village                 |
|263         |Yorkville West       |107         |Gramercy                     |
|239         |Upper West Side South|41          |Central Harlem               |
|162         |Midtown East         |141         |Lenox Hill West              |
|137         |Kips Bay             |107         |Gramercy                     |
|68          |East Chelsea         |158 

In [17]:
from pyspark.sql.functions import sum as fsum, when

null_zones = df_enriched.select(
    fsum(when(col("pickup_zone").isNull(), 1).otherwise(0)).alias("pickup_zone_nulls"),
    fsum(when(col("dropoff_zone").isNull(), 1).otherwise(0)).alias("dropoff_zone_nulls"),
)

null_zones.show()

+-----------------+------------------+
|pickup_zone_nulls|dropoff_zone_nulls|
+-----------------+------------------+
|                0|                 0|
+-----------------+------------------+



In [18]:
from pyspark.sql.functions import input_file_name

df_all_with_source = df_all.withColumn("source_file", input_file_name())

In [19]:
from pyspark.sql.functions import col

df_clean = df_all_with_source.filter(
    (col("passenger_count") > 0) &
    (col("trip_distance") > 0) &
    (col("tpep_dropoff_datetime") > col("tpep_pickup_datetime"))
)

df_dedup = df_clean.dropDuplicates([
    "VendorID",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID"
])


In [20]:
from pyspark.sql.functions import broadcast

zones_pickup = zones.select(
    col("LocationID").alias("PULocationID"),
    col("Zone").alias("pickup_zone")
)

zones_dropoff = zones.select(
    col("LocationID").alias("DOLocationID"),
    col("Zone").alias("dropoff_zone")
)

df_enriched = (
    df_dedup
    .join(broadcast(zones_pickup), "PULocationID", "left")
    .join(broadcast(zones_dropoff), "DOLocationID", "left")
)

In [21]:
from pyspark.sql.functions import unix_timestamp, to_date, current_timestamp

df_ready = (
    df_enriched
    .withColumn(
        "trip_duration_minutes",
        (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60.0
    )
    .withColumn("pickup_date", to_date("tpep_pickup_datetime"))
    .withColumn("ingested_at", current_timestamp())
)


In [22]:
df_ready.select(
    "tpep_pickup_datetime", "tpep_dropoff_datetime",
    "PULocationID", "pickup_zone",
    "DOLocationID", "dropoff_zone",
    "passenger_count", "trip_distance",
    "trip_duration_minutes", "pickup_date",
    "source_file", "ingested_at"
).show(5, truncate=False)


+--------------------+---------------------+------------+-------------------------+------------+--------------+---------------+-------------+---------------------+-----------+-------------------------------------------------------------------+--------------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|pickup_zone              |DOLocationID|dropoff_zone  |passenger_count|trip_distance|trip_duration_minutes|pickup_date|source_file                                                        |ingested_at               |
+--------------------+---------------------+------------+-------------------------+------------+--------------+---------------+-------------+---------------------+-----------+-------------------------------------------------------------------+--------------------------+
|2025-01-01 03:38:57 |2025-01-01 04:05:11  |230         |Times Sq/Theatre District|1           |Newark Airport|4              |17.91        |26.233333333333334   |2025-01-01 |file:///home

In [23]:
import os, json, shutil
from datetime import datetime

from pyspark.sql.functions import (
    col, input_file_name, current_timestamp, to_date, unix_timestamp, broadcast
)

# Paths (dentro do container)
INBOX = "/home/jovyan/work/data/inbox"
LOOKUP = "/home/jovyan/work/data/taxi_zone_lookup.parquet"
OUTBOX = "/home/jovyan/work/data/outbox/trips_enriched.parquet"
STATE = "/home/jovyan/work/state/manifest.json"

os.makedirs("/home/jovyan/work/state", exist_ok=True)
os.makedirs("/home/jovyan/work/data/outbox", exist_ok=True)

def load_manifest(path: str):
    if not os.path.exists(path):
        return {"processed_files": []}
    with open(path, "r") as f:
        return json.load(f)

def save_manifest(path: str, manifest: dict):
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(manifest, f, indent=2)
    os.replace(tmp, path)  # escrita “atômica” (evita manifest corrompido)

def outbox_has_data(path: str) -> bool:
    return (
        os.path.exists(path)
        and os.path.isdir(path)
        and any(name.startswith("part-") and name.endswith(".parquet") for name in os.listdir(path))
    )

# (Opcional) se existir outbox vazio/corrompido, remove antes (evita UNABLE_TO_INFER_SCHEMA)
if os.path.exists(OUTBOX) and not outbox_has_data(OUTBOX):
    print("⚠️ OUTBOX exists but has no parquet parts. Deleting corrupted/empty outbox folder...")
    shutil.rmtree(OUTBOX)

# 1) Descobrir arquivos novos
manifest = load_manifest(STATE)
processed = set(x["filename"] for x in manifest["processed_files"])

all_files = sorted([f for f in os.listdir(INBOX) if f.endswith(".parquet")])
new_files = [f for f in all_files if f not in processed]

print("All files:", all_files)
print("New files:", new_files)

if not new_files:
    print("✅ Nothing new to process. Exiting without changes.")
else:
    now = datetime.utcnow().isoformat()

    # 2) Ler SOMENTE os novos arquivos
    new_paths = [os.path.join(INBOX, f) for f in new_files]
    df_new = spark.read.parquet(*new_paths).withColumn("source_file", input_file_name())

    # 3) Limpeza
    df_clean = df_new.filter(
        (col("passenger_count") > 0) &
        (col("trip_distance") > 0) &
        (col("tpep_dropoff_datetime") > col("tpep_pickup_datetime"))
    )

    # 4) Dedup (nos novos)
    dedup_key = ["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID"]
    df_dedup = df_clean.dropDuplicates(dedup_key)

    # 5) Lookup + joins (broadcast pois lookup é pequeno)
    zones = spark.read.parquet(LOOKUP)

    zones_pickup = zones.select(
        col("LocationID").alias("PULocationID"),
        col("Zone").alias("pickup_zone")
    )

    zones_dropoff = zones.select(
        col("LocationID").alias("DOLocationID"),
        col("Zone").alias("dropoff_zone")
    )

    df_enriched = (
        df_dedup
        .join(broadcast(zones_pickup), "PULocationID", "left")
        .join(broadcast(zones_dropoff), "DOLocationID", "left")
    )

    # 6) Derivadas + metadata
    df_ready = (
        df_enriched
        .withColumn(
            "trip_duration_minutes",
            (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60.0
        )
        .withColumn("pickup_date", to_date("tpep_pickup_datetime"))
        .withColumn("ingested_at", current_timestamp())
    )

    # 7) Escrever APENAS o que é novo (APPEND) — mais leve, evita crash da JVM
    # (Dica: coalesce reduz número de arquivos de saída e pressão de memória)
    df_ready.coalesce(4).write.mode("append").parquet(OUTBOX)

    # 8) Atualizar manifest (idempotência)
    # (opcional) salvar também rows após clean+dedup por arquivo — mas sem collect pesado
    # aqui vamos salvar só o filename e timestamp (mínimo exigido)
    for f in new_files:
        manifest["processed_files"].append({
            "filename": f,
            "processed_at": now
        })

    save_manifest(STATE, manifest)

    print("✅ Run complete!")
    print("Output path:", OUTBOX)
    print("Manifest updated:", STATE)


All files: ['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet']
New files: []
✅ Nothing new to process. Exiting without changes.


In [24]:
import os, shutil

OUTBOX = "/home/jovyan/work/data/outbox/trips_enriched.parquet"

print("OUTBOX exists?", os.path.exists(OUTBOX))
if os.path.exists(OUTBOX):
    print("OUTBOX contents:", os.listdir(OUTBOX)[:50])

# Se existir mas não tiver part-*.parquet, apaga
def outbox_has_data(path: str) -> bool:
    return (
        os.path.exists(path)
        and os.path.isdir(path)
        and any(name.startswith("part-") and name.endswith(".parquet") for name in os.listdir(path))
    )

if os.path.exists(OUTBOX) and not outbox_has_data(OUTBOX):
    print("⚠️ OUTBOX exists but has no parquet parts. Deleting corrupted/empty outbox folder...")
    shutil.rmtree(OUTBOX)
    print("✅ Deleted. Now OUTBOX exists?", os.path.exists(OUTBOX))


OUTBOX exists? True
OUTBOX contents: ['.part-00000-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet.crc', '.part-00001-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet.crc', '.part-00002-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet.crc', '.part-00003-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet.crc', '._SUCCESS.crc', 'part-00000-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet', 'part-00001-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet', 'part-00002-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet', 'part-00003-3f0416c5-0c6e-4dbc-924b-9db42d02f3d4-c000.snappy.parquet', '_SUCCESS']


In [25]:
spark.read.parquet("/home/jovyan/work/data/outbox/trips_enriched.parquet").count()

5486133